### 결측치의 생성과 이해

결측치는 기본적으로 NumPy의 NaN, Pandas의 NA가 있으며 이와 유사한 것으로 None이 있다.

In [1]:
import numpy as np
import pandas as pd
np.nan, pd.NA

(nan, <NA>)

결측치 관련 원소를 기반으로 비교연산을 실시할 수 있으나 에러가 발생하지 않지만 만족스러운 결과가 나오지 않기 때문에 관련 코드를 별도의 필터링 등에 활용할 수 없다. 그리하여 결측치 관련 원소를 다루기 위해서는 전용 함수 또는 메서드를 사용해야 한다.

In [2]:
np.nan == np.nan, pd.NA == pd.NA

(False, <NA>)

### 결측치의 식별

In [3]:
ser1 = pd.Series([1, 3, pd.NA, 5])
ser1

0       1
1       3
2    <NA>
3       5
dtype: object

In [4]:
ser1.isna() # 결측값이 True

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
ser1.isnull() # 결측값이 True

0    False
1    False
2     True
3    False
dtype: bool

In [6]:
ser1.notna() # 관측값이 True

0     True
1     True
2    False
3     True
dtype: bool

In [7]:
ser1.notnull() # 관측값이 True

0     True
1     True
2    False
3     True
dtype: bool

상기 결측/관측 처리 메서드 .isna(), .isnull(), .notna(), .notnull() 을 활용하여 결측/관측치를 필터링 할 수 있음. 그리고 .isna()와 .isnull()은 동작이 같고, .notna()와 .notnull() 또한 서로 동작이 같아 사용자 입맛에 맞는 메서드를 사용하면 된다.

In [8]:
ser1.isna().sum()

1

.isna() 등 결측/관측 식별 메서드의 결과가 True/False인 점을 활용하여 .sum() 메서드로 Series 객체의 결측치 개수를 확인할 수 있다. 그리고 .isna() 관련 메서드를 DataFrame 객체에서 사용할 경우 모든 원소에 대해 결측/관측 검사를 실시하고 True/False 를 반환한다.

In [9]:
df_na = pd.DataFrame({"var1": [1, 2, pd.NA],
                      "var2": [100, pd.NA, pd.NA]})
df_na

,var1,var2
0,1,100
1,2,<NA>
2,<NA>,<NA>


In [10]:
df_na.isna()

,var1,var2
0,False,False
1,False,True
2,True,True


In [11]:
df_na.isna().sum()

var1    1
var2    2
dtype: int64

결측치가 .isna() 같은 메서드로 식별이 불가능한 빈칸, 하이픈 등 특수한 값으로 기록되어있는 경우 비교연산 관련 연산자나 메서드를 사용해야함.

In [12]:
df_sna = pd.DataFrame({"col1": [11, 22, "-"],
                       "col2": ["x", 30, "x"]})
df_sna

,col1,col2
0,11,x
1,22,30
2,-,x


In [13]:
sum(df_sna["col1"] == "-"), (df_sna["col1"] == "-").sum()

(1, 1)

In [14]:
df_sna.apply(lambda x: x.isin(["-", "x"]).sum())

col1    1
col2    2
dtype: int64

In [15]:
def na_counter(x, na_type):
    return x.isin(na_type).sum()

df_sna.apply(na_counter, na_type = ["-", "x"])

col1    1
col2    2
dtype: int64

### 결측치의 필터링

In [16]:
ser1[ser1.isna()]

2    <NA>
dtype: object

In [17]:
ser1[ser1.notna()]

0    1
1    3
3    5
dtype: object

In [18]:
ser1[~ser1.isna()] # ~ 표시는 True/False를 반전시킴

0    1
1    3
3    5
dtype: object

### 결측치의 제거

Series 객체의 경우 .notna() 메서드를 사용한 필터링 보다 .dropna() 를 사용한 결측치 제거가 보다 간결하다. 단, DataFrame객체의 경우 특정 변수에 있는 결측치를 기준으로 필터링 할 때에는 .isna() 또는 .notna()가 필요하며 .dropna() 는 결측치를 일괄(전체 변수)로 다루는 경우 사용된다.

In [19]:
ser2 = ser1[ser1.notna()]
ser2

0    1
1    3
3    5
dtype: object

In [20]:
ser3 = ser1.dropna()
ser3

0    1
1    3
3    5
dtype: object

In [21]:
df_na.dropna()

,var1,var2
0,1,100


In [22]:
# how = "all": 특정 행의 원소가 모두 결측값으로 구성되어 있는 경우 해당 행 제거
df_na.dropna(how = "all") 

,var1,var2
0,1,100
1,2,<NA>


In [23]:
df_not_na = df_na.dropna()
df_not_na

,var1,var2
0,1,100


In [24]:
df_na.loc[df_na["var1"].notna(), ]

,var1,var2
0,1,100
1,2,<NA>


### 결측치의 대치(imputation)

결측치의 대치는 .fillna() 같은 결측치 대치 전용 메서드를 사용하거나 .isna() 같은 결측치 식별 메서드를 활용한 방법이 있을 수 있다.

In [25]:
ser1.fillna(0)

0    1
1    3
2    0
3    5
dtype: int64

In [26]:
ser4 = ser1.fillna(0)
ser4

0    1
1    3
2    0
3    5
dtype: int64

In [27]:
ser5 = ser1.fillna(ser1.mean()) # 평균값 대치
ser5

0    1.0
1    3.0
2    3.0
3    5.0
dtype: float64

시계열 데이터의 경우 결측치 발생 이전의 데이터 또는 결측치 발생 이후의 데이터를 끌어와서 채워넣어야 하는 경우가 있다. 이를 위해 .fillna() 메서드에서는 "method" 인자를 제공하며 여기서 "ffill" 또는 "bfill"을 사용할 수 있다.

In [28]:
ser1.fillna(method = "ffill")

0    1
1    3
2    3
3    5
dtype: int64

In [29]:
ser1.fillna(method = "bfill")

0    1
1    3
2    5
3    5
dtype: int64

In [30]:
ser1

0       1
1       3
2    <NA>
3       5
dtype: object

In [31]:
# ser1.interpolate() # pd.NA로 생성된 결측은 보간(interpolate)을 지원하지 않음

In [32]:
ser6 = pd.Series([0, 1, np.nan, 3])
ser6.interpolate()

0    0.0
1    1.0
2    2.0
3    3.0
dtype: float64